### stage dataset

build a stage dataset with feature extraction

+ add neighbors information as features


# Imports

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# to use only one GPU.
# use this on r-001
# otherwise comment
import os
os.environ["CUDA_VISIBLE_DEVICES"]="7"


## Load Data

In [2]:
import os
os.getcwd()


'/scratch/hpda/indycar/predictor/notebook/5.DeployModel'

In [3]:
#
# parameters
#
#year = '2017'
year = '2018'
#event = 'Toronto'
events = ['Phoenix','Indy500','Texas','Iowa','Pocono','Gateway']


In [4]:
# make indy car completed_laps dataset
# car_number, completed_laps, rank, elapsed_time, rank_diff, elapsed_time_diff 
def make_cl_data(dataset):

    # pick up data with valid rank
    rankdata = dataset.rename_axis('MyIdx').sort_values(by=['elapsed_time','MyIdx'], ascending=True)
    rankdata = rankdata.drop_duplicates(subset=['car_number', 'completed_laps'], keep='first')

    # resort by car_number, lap
    uni_ds = rankdata.sort_values(by=['car_number', 'completed_laps', 'elapsed_time'], ascending=True)    
    uni_ds = uni_ds.drop(["unique_id", "best_lap", "current_status", "track_status", "lap_status",
                      "laps_behind_leade","laps_behind_prec","overall_rank","pit_stop_count",
                      "last_pitted_lap","start_position","laps_led"], axis=1)
    
    carnumber = set(uni_ds['car_number'])
    print('cars:', carnumber)
    print('#cars=', len(carnumber))
   
    # faster solution , uni_ds already sorted by car_number and lap
    uni_ds['rank_diff'] = uni_ds['rank'].diff()
    mask = uni_ds.car_number != uni_ds.car_number.shift(1)
    uni_ds['rank_diff'][mask] = 0
    
    uni_ds['time_diff'] = uni_ds['elapsed_time'].diff()
    mask = uni_ds.car_number != uni_ds.car_number.shift(1)
    uni_ds['time_diff'][mask] = 0
    
    df = uni_ds[['car_number','completed_laps','rank','elapsed_time','rank_diff','time_diff']]
    
    return df

def make_lapstatus_data(dataset):
    final_lap = max(dataset.completed_laps)
    total_laps = final_lap + 1

    # get records for the cars that finish the race
    completed_car_numbers= dataset[dataset.completed_laps == final_lap].car_number.values
    completed_car_count = len(completed_car_numbers)

    print('count of completed cars:', completed_car_count)
    print('completed cars:', completed_car_numbers)
    
    #pick up one of them
    onecar = dataset[dataset['car_number']==completed_car_numbers[0]]
    onecar = onecar.drop_duplicates(subset=['car_number', 'completed_laps'], keep='first')
    return onecar[['completed_laps','track_status']]
    

In [5]:
def load_data(event, year):
    inputfile = '../data/final/C_'+ event +'-' + year + '-final.csv'
    outputprefix = year +'-' + event + '-'
    dataset = pd.read_csv(inputfile)
    #dataset.info(verbose=True)    
    
    final_lap = max(dataset.completed_laps)
    total_laps = final_lap + 1

    # get records for the cars that finish the race
    completed_car_numbers= dataset[dataset.completed_laps == final_lap].car_number.values
    completed_car_count = len(completed_car_numbers)

    print('count of completed cars:', completed_car_count)
    print('completed cars:', completed_car_numbers)

    #make a copy
    alldata = dataset.copy()
    dataset = dataset[dataset['car_number'].isin(completed_car_numbers)]
    rankdata = alldata.rename_axis('MyIdx').sort_values(by=['elapsed_time','MyIdx'], ascending=True)
    rankdata = rankdata.drop_duplicates(subset=['car_number', 'completed_laps'], keep='first')
    
    cldata = make_cl_data(dataset)
    flagdata = make_lapstatus_data(dataset)
    acldata = make_cl_data(alldata)

    return alldata, rankdata, acldata, flagdata

### overall view of laptime scatter plots



In [6]:
def get_cardata(curcarno, ycol='time_diff'):
    car = acldata[acldata['car_number']==curcarno]
    #print(car['time_diff'].describe())
    
    cols=['completed_laps','rank','car_number','lap_status','track_status',
    'pit_stop_count','current_status','start_position']
    colid={key:idx for idx, key in enumerate(cols)}

    cardata = rankdata[rankdata['car_number'] == curcarno]

    carstatus = [[row[0], row[1],row[2],row[3],row[4],row[5],row[6],row[7]] for row in cardata[
        ['completed_laps','rank','car_number','lap_status','track_status',
        'pit_stop_count','current_status','start_position']].values]
    
    x = car['completed_laps'][1:].values
    y = car[ycol][1:].values

    pits=[]
    yellowflags=[]
    lastflag = 'x'
    for row in carstatus:
        lap = int(row[colid['completed_laps']])

        if row[colid['lap_status']]=='P':
            pits.append(lap)

        if row[colid['track_status']]=='Y':
            if lastflag != 'Y':       
                #start
                yellowflags.append(lap)
        else:
            if lastflag == 'Y':       
                #end
                yellowflags.append(lap)        
        lastflag = row[colid['track_status']]

    #pit lap
    pits = np.array(pits)
    #start, end lap
    #
    yellowflags = np.array(yellowflags)
    if (yellowflags.shape[0] % 2)==1:
        print('crash?:carno=', curcarno)
        yellowflags = []
    else:
        yellowflags = np.array(yellowflags).reshape((-1,2))    
    
    return car, x, y, pits, yellowflags

In [7]:
def draw_laptime(ax, ycol='LapTime'):
    
    ymax = np.max(y)
    
    #yellow flags
    for yflag in yellowflags:
        ax.bar(yflag[0],ymax, width = yflag[1] - yflag[0], align='edge', color = 'y', zorder=-100)

    #plt.plot(np.arange(laptime.shape[0]), laptime, marker='.')
    ax.scatter(x, y, marker='.')    

    #pit stops    
    ax.scatter(pits, y[pits-1], marker='x', color='r')
    for pit in pits:
        ax.bar(pit,height = y[pits-1], width = .8, align='center', color = 'r', zorder=-100)
    
    #ax.set_xlim((0, 33))
    #ax.set_ylim((0, 201))
    ax.set_xlabel('Lap')
    ax.set_ylabel(ycol)
    
def draw_pitstop(ax, pits):
    #pit stops    
    ax.scatter(pits, y[pits-1], marker='x', color='r')
    for pit in pits:
        ax.bar(pit,height = y[pits-1], width = .8, align='center', color = 'r', zorder=-100)

    
def draw_yellowflag(ax,yellowflags):
    ymax = np.max(y)
    #yellow flags
    for yflag in yellowflags:
        ax.bar(yflag[0],ymax, width = yflag[1] - yflag[0], align='edge', color = 'y', zorder=-100)

### bulid the dataset

In [8]:
def get_stagedata(TRIM=2):
    """
    input: acldata, rankdata, the whole dataset
    output: stagedata['car_number','stage','start_lap','end_lap','start_rank','end_rank']
    """
    carlist = set(acldata['car_number'])
    data = []
    for curcarno in carlist:
        car = acldata[acldata['car_number']==curcarno]
        #print(car['time_diff'].describe())

        cols=['completed_laps','rank','car_number','lap_status','track_status',
        'pit_stop_count','current_status','start_position']
        colid={key:idx for idx, key in enumerate(cols)}

        cardata = rankdata[rankdata['car_number'] == curcarno]

        carstatus = [[row[0], row[1],row[2],row[3],row[4],row[5],row[6],row[7]] for row in cardata[
            ['completed_laps','rank','car_number','lap_status','track_status',
            'pit_stop_count','current_status','start_position']].values]

        pits=[]
        yellowflags=[]
        lastflag = 'x'
        
        stage = 0
        start_lap = 0
        pit_lap = 0
        pit_in_caution = 0
        for row in carstatus:
            lap = int(row[colid['completed_laps']])
            
            if lap==0:
                #hex2int
                start_rank = int(row[colid['start_position']], 16)     
            
            if row[colid['lap_status']]=='P':
                pits.append(lap)
                end_lap = lap - TRIM
                end_rank = carstatus[end_lap][colid['rank']]
                #add one record
                #stagedata['car_number','stage','start_lap','end_lap','start_rank','end_rank']
                data.append([curcarno,stage,start_lap, end_lap, pit_lap, pit_in_caution, start_rank, end_rank])
                # prepare for the next
                stage += 1
                start_lap = end_lap  # end_lap + 1?
                start_rank = end_rank
                pit_lap = lap  #current is pit lap
                pit_in_caution = 1 if row[colid['track_status']]=='Y' else 0
                

            if row[colid['track_status']]=='Y':
                if lastflag != 'Y':       
                    #start
                    yellowflags.append(lap)
            else:
                if lastflag == 'Y':       
                    #end
                    yellowflags.append(lap)        
            lastflag = row[colid['track_status']]

        #final
        end_lap = carstatus[-1][colid['completed_laps']]
        end_rank = carstatus[-1][colid['rank']]
        #add one record
        #stagedata['car_number','stage','start_lap','end_lap',pit_lap, pit_in_caution,'start_rank','end_rank']
        data.append([curcarno,stage,start_lap, end_lap, pit_lap, pit_in_caution, start_rank, end_rank])
        
        #pit lap
        pits = np.array(pits)
        #start, end lap
        #
        yellowflags = np.array(yellowflags)
        if (yellowflags.shape[0] % 2)==1:
            print('crash?:carno=', curcarno)
            yellowflags = []
        else:
            yellowflags = np.array(yellowflags).reshape((-1,2))  
            
    data = np.array(data)
    #['car_number','stage','start_lap','end_lap',pit_lap, pit_in_caution,'start_rank','end_rank']
    df = pd.DataFrame({'car_number': data[:, 0], 'stage': data[:, 1],'start_lap': data[:, 2],
                      'end_lap': data[:, 3], 'pit_lap': data[:, 4], 'pit_in_caution': data[:, 5],
                       'start_rank': data[:, 6],'end_rank': data[:, 7]})
    return df, data

### events

In [9]:
def extract_features(stagedata):
    """
    input: (alldata, rankdata, acldata, flagdata, stage, data)
    
    target
    eventid
    car_number
    stageid
    
    #0, gobal info 
    firststage  ;  1/0    
    pit_in_caution; 1/0
    start_position  ; #
    
    # 0 order of #rank
    start_rank  ;  #rank
    start_rank_ratio; ; #rank/carnum 
    top_pack    ;  top5 1/0 
    bottom_pack  ; bottom5 1/0 
    
    average_rank  ; previous stage
    average_rank_all  ; all previous stages
    
    # 1 order of #rank
    change_in_rank; previous stage
    change_in_rank_all;  all previous stages
    # 2 order of #rank
    rate_of_change; previous stage
    rate_of_change_all;  all previous stages
    
    #neighbors;   
    prev_nb0_change_in_rank
    prev_nb1_change_in_rank
    prev_nb2_change_in_rank
    follow_nb0_change_in_rank
    follow_nb1_change_in_rank
    follow_nb2_change_in_rank
    
    # more features
    laptime_green_mean_prev  ; mean and std of the laptime in green laps of previous stage
    laptime_green_std_prev
    laptime_green_mean_all  ; mean and std of the laptime in green laps before
    laptime_green_std_all
    
    laptime_mean_prev  ; mean and std of the laptime in all laps of previous stage
    laptime_std_prev
    laptime_mean_all  ; mean and std of the laptime in all laps before
    laptime_std_all
    
    laps_prev   ; lap number of the previous stage
    laps_after_last_pitstop  ; lap number after the last pitstop
    
    pittime_prev   ; pittime of previous pitstop
    
    
    """
    eventid = 0
    cols=['target','start_lap',
             'eventid','car_number','stageid',
             'firststage','pit_in_caution','start_position',
             'start_rank','start_rank_ratio','top_pack','bottom_pack',
             'average_rank','average_rank_all',
             'change_in_rank','change_in_rank_all','rate_of_change','rate_of_change_all',
             'laptime_green_mean_prev','laptime_green_std_prev','laptime_green_mean_all','laptime_green_std_all', 
             'laptime_mean_prev','laptime_std_prev','laptime_mean_all','laptime_std_all', 
             'laps_prev','laps_after_last_pitstop','pittime_prev',     
             'prev_nb0_change_in_rank','prev_nb1_change_in_rank','prev_nb2_change_in_rank',
             'follow_nb0_change_in_rank','follow_nb1_change_in_rank','follow_nb2_change_in_rank']
    colid={key:idx for idx, key in enumerate(cols)}
    
    datacols = ['car_number','stage','start_lap','end_lap','pit_lap', 'pit_in_caution', 'start_rank','end_rank']
    datacolid={key:idx for idx, key in enumerate(datacols)}
    
    #maintain a <car,lap, (rank, stage)> index 
    idxcols = ['car_number','completed_laps','rank','stageid']
    carindex = {}
    
    stagedf = pd.DataFrame([],columns=cols)
    for event in stagedata.keys():
        alldata, rankdata, acldata, flagdata, stage, data = stagedata[event]
        carlist = set(acldata['car_number'])
        laplist = set(acldata['completed_laps'])
        totalcars = len(carlist)
        totallaps = len(laplist)

        # init the index
        #carindex[eventid] = np.array((totalcars, totallaps, 2))
        carindex[eventid] = pd.DataFrame([],columns=idxcols)
        
        records = []
        
        laphist = []
        rankhist = []
        rankchghist = []        
        #['car_number','stage','start_lap','end_lap',pit_lap, pit_in_caution, 'start_rank','end_rank']
        for row in data:
            # a new record
            rec={}
            #status, 'start_lap', start a new car
            if row[datacolid['start_lap']] == 0:
                start_position = row[datacolid['start_rank']]
                laphist = []
                rankhist = []
                rankchghist = []
            
            #target 
            rec['target'] = row[datacolid['end_rank']] - row[datacolid['start_rank']]
            rec['start_lap'] = row[datacolid['start_lap']]
            
            #features
            rec['eventid'] = eventid
            rec['car_number'] = row[datacolid['car_number']]
            rec['stageid'] = row[datacolid['stage']]
            rec['firststage'] = 0 if row[datacolid['stage']]==0 else 1
            rec['pit_in_caution'] = row[datacolid['pit_in_caution']]
            rec['start_position'] = start_position
            rec['start_rank'] = row[datacolid['start_rank']]
            rec['start_rank_ratio'] = rec['start_rank'] / totalcars
            rec['top_pack'] = 1 if rec['start_rank']<=5 else 0
            #todo, not accurate here
            rec['bottom_pack'] = 1 if rec['start_rank']>=totalcars-5 else 0
            #get from laphist
            if laphist:
                #previous stage
                prev_stage = laphist[-1]
                rec['average_rank'] = np.mean(list(prev_stage['rank']))
                rec['average_rank_all'] = np.mean(rankhist)
            else:
                rec['average_rank'] = rec['start_rank']
                rec['average_rank_all'] = rec['start_rank']
                
            #change in rank
            if rankchghist:
                rec['change_in_rank'] = rankchghist[-1]
                rec['change_in_rank_all'] = np.mean(rankchghist)
            else:
                rec['change_in_rank'] = 0
                rec['change_in_rank_all'] = 0
                
            #rate of change in rank
            if len(rankchghist)>2:
                rec['rate_of_change'] = rankchghist[-1] - rankchghist[-2]
                rec['rate_of_change_all'] = (rankchghist[-1] - rankchghist[0])/(len(rankchghist)-1)
            else:
                rec['rate_of_change'] = 0
                rec['rate_of_change_all'] = 0            
                
            # init neighbor info as 0
            rec['prev_nb0_change_in_rank'] = 0
            rec['prev_nb1_change_in_rank'] = 0
            rec['prev_nb2_change_in_rank'] = 0
            rec['follow_nb0_change_in_rank'] = 0
            rec['follow_nb1_change_in_rank'] = 0
            rec['follow_nb2_change_in_rank'] = 0
            
            # laptime
            if not laphist:
                rec['laptime_green_mean_prev'] = 0
                rec['laptime_green_std_prev'] = 0
                rec['laptime_green_mean_all'] = 0
                rec['laptime_green_std_all'] = 0
                rec['laptime_mean_prev'] = 0
                rec['laptime_std_prev'] = 0
                rec['laptime_mean_all'] = 0
                rec['laptime_std_all'] = 0
                rec['laps_prev'] = 0
                rec['laps_after_last_pitstop'] = 0
                rec['pittime_prev'] = 0
            else:
                #previous stage greeds
                #lapdata = rankdata[(rankdata['car_number']== rec['car_number']) & 
                #               (rankdata['completed_laps']>=row[datacolid['start_lap']]) &
                #               (rankdata['completed_laps']<row[datacolid['end_lap']])
                #              ]                
                prev_stage = laphist[-1]
                #green_laps = list(prev_stage[prev_stage['track_status']=='G']['last_laptime'])
                green_laps = prev_stage[prev_stage['track_status']=='G']['completed_laps'].values
                green_laps = list(acldata[(acldata['car_number']== rec['car_number']) & 
                        (acldata['completed_laps'].isin(green_laps))]['time_diff'])
                
                
                rec['laptime_green_mean_prev'] = np.mean(green_laps)
                rec['laptime_green_std_prev'] =  np.std(green_laps)
                #all_laps = list(prev_stage['last_laptime'])
                all_laps = prev_stage['completed_laps'].values
                all_laps = list(acldata[(acldata['car_number']== rec['car_number']) & 
                        (acldata['completed_laps'].isin(all_laps))]['time_diff'])
                
                
                rec['laptime_mean_prev'] = np.mean(all_laps)
                rec['laptime_std_prev'] =  np.std(all_laps)
                rec['laps_prev'] = len(all_laps)
                
                last_pit_lap = int(prev_stage['last_pitted_lap'].values[-1], 16) 
                rec['laps_after_last_pitstop'] = rec['start_lap'] - last_pit_lap
                
                #get two laps
                #pit_data = rankdata[(rankdata['car_number']== rec['car_number']) & 
                #               (rankdata['completed_laps']>=last_pit_lap) &
                #               (rankdata['completed_laps']<last_pit_lap+2)
                #              ]
                #rec['pittime_prev'] = np.mean(list(pit_data['last_laptime']))
                pit_data = acldata[(acldata['car_number']== rec['car_number']) & 
                               (acldata['completed_laps']>=last_pit_lap) &
                               (acldata['completed_laps']<last_pit_lap+2)
                              ]
                rec['pittime_prev'] = np.mean(list(pit_data['time_diff']))
                                
                
                # all previous laps laptime
                all_data = rankdata[(rankdata['car_number']== rec['car_number']) & 
                               (rankdata['completed_laps']<rec['start_lap'])
                              ]                
                green_laps = all_data[all_data['track_status']=='G']['completed_laps'].values
                green_laps = list(acldata[(acldata['car_number']== rec['car_number']) & 
                        (acldata['completed_laps'].isin(green_laps))]['time_diff'])
                
                rec['laptime_green_mean_all'] = np.mean(green_laps)
                rec['laptime_green_std_all'] =  np.std(green_laps)
                #all_laps = list(all_data['last_laptime'])
                all_laps = all_data['completed_laps'].values
                all_laps = list(acldata[(acldata['car_number']== rec['car_number']) & 
                        (acldata['completed_laps'].isin(all_laps))]['time_diff'])                
                rec['laptime_mean_all'] = np.mean(all_laps)
                rec['laptime_std_all'] =  np.std(all_laps)                
            
            #add one record
            records.append([rec[col] for col in cols])
            
            #update for the new stage
            lapdata = rankdata[(rankdata['car_number']== rec['car_number']) & 
                               (rankdata['completed_laps']>=row[datacolid['start_lap']]) &
                               (rankdata['completed_laps']<row[datacolid['end_lap']])
                              ]
            laphist.append(lapdata)
            rankhist.extend(list(lapdata['rank']))
            rankchghist.append(row[datacolid['end_rank']] - row[datacolid['start_rank']])
            
            # add to index, go through the lapdata
            #or _lap in range(row[datacolid['start_lap']],row[datacolid['end_lap']]):
            for index, row in lapdata.iterrows():
                #carindex[eventid][rec['car_number'], row['completed_laps'], 0] = row['rank']
                #carindex[eventid][rec['car_number'], row['completed_laps'], 1 = rec['stageid']
                laprec = [rec['car_number'], row['completed_laps'], row['rank'],rec['stageid']]
                df = pd.DataFrame([laprec],columns=idxcols)
                carindex[eventid] = pd.concat([carindex[eventid], df], ignore_index=True)
            
          
        # push this event into stage dataframe
        print('eventid=%d, records=%d'%(eventid, len(records)))
        df = pd.DataFrame(records,columns=cols)
        stagedf = pd.concat([stagedf, df], ignore_index=True)
        
        #for the next new event
        eventid += 1
        
    return stagedf, carindex

def update_neighbor(stagedf, carindex):
    # go through stage data to add neighbor info
    # carindex[eventid] <car, lap, (rank, stage)
    ndf = stagedf.copy()
    
    #stagedf.index = pd.RangeIndex(len(stagedf.index))
        
    for index, row in stagedf.iterrows():
        
        eventid = row['eventid']
        
        currank = row['start_rank']
        curlap = row['start_lap']
        if curlap == 0:
            continue
        
        df = carindex[row['eventid']]
        df_lap = df[df['completed_laps']==curlap]
        prev0 = df_lap[df_lap['rank'] == currank - 1]
        prev1 = df_lap[df_lap['rank'] == currank - 2]
        prev2 = df_lap[df_lap['rank'] == currank - 3]
        follow0 = df_lap[df_lap['rank'] == currank + 1]
        follow1 = df_lap[df_lap['rank'] == currank + 2]
        follow2 = df_lap[df_lap['rank'] == currank + 3]
        
        recs = [prev0, prev1, prev2, follow0, follow1,follow2]
        recloc = ['prev_nb0_change_in_rank','prev_nb1_change_in_rank','prev_nb2_change_in_rank',
                 'follow_nb0_change_in_rank','follow_nb1_change_in_rank','follow_nb2_change_in_rank']
        for recidx, rec in enumerate(recs):
            if rec.empty:
                continue
            
            #fail on eventid=5, trim=0, duplicate rank appears rank=19
            rec = rec.iloc[:1]
            
            car_number = rec['car_number']
            stageid = rec['stageid']
            #print('event:%d, car:%d, stage:%d, index:%d'%(eventid, car_number, stageid, index))
            #find the record of prev0 by eventid, carno, stageid
            #neighbor = stagedf[(stagedf['eventid'] == eventid) & (stagedf['car_number'] == car_number) & (stagedf['stageid'] == stageid)]
            #neighbor = ndf[(ndf['eventid'] == eventid) & (ndf['car_number'] == car_number) & (ndf['stageid'] == stageid)]
            neighbor = ndf.query('eventid == %d & car_number==%d & stageid==%d'%(eventid,car_number,stageid))
            
            if neighbor.empty:
                print('error: empty neighbor at: '%(eventid,car_number,stageid))
                continue
            
            # update current row
            #print('index=%d, col=%s'%(index, recloc[recidx]))
            stagedf.loc[index,recloc[recidx]] = neighbor['change_in_rank'].values[0]
            #stagedf.loc[index,recloc[recidx]] = 1
                                
    return stagedf

### load data

In [10]:
stagedata = {}
traindata = None
for event in events:
    #alldata, rankdata, acldata, flagdata
    stagedata[event] = load_data(event, year)
    
    alldata, rankdata, acldata, flagdata = stagedata[event]
    carlist = set(acldata['car_number'])
    laplist = set(acldata['completed_laps'])
    print('%s: carno=%d, lapnum=%d'%(event, len(carlist), len(laplist)))
    
    stage, data = get_stagedata(0)
    stagedata[event] = (alldata, rankdata, acldata, flagdata, stage, data)
    
    if traindata is None:
        traindata = data
    else:
        traindata = np.vstack((traindata, data))

count of completed cars: 11
completed cars: [ 1  6 27  9 28  5 20 14 15 22 30]
cars: {1, 5, 6, 9, 14, 15, 20, 22, 27, 28, 30}
#cars= 11
count of completed cars: 11
completed cars: [ 1  6 27  9 28  5 20 14 15 22 30]
cars: {1, 4, 5, 6, 9, 10, 12, 14, 15, 18, 19, 20, 21, 22, 23, 26, 27, 28, 30, 32, 59, 88, 98}
#cars= 23


/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Phoenix: carno=23, lapnum=251
crash?:carno= 10
crash?:carno= 19
count of completed cars: 18
completed cars: [12 20  9 27 28 22 29  1  6 15 66 98  4 88 25 60 64 23]
cars: {64, 1, 66, 98, 4, 6, 9, 12, 60, 15, 20, 22, 23, 88, 25, 27, 28, 29}
#cars= 18
count of completed cars: 18
completed cars: [12 20  9 27 28 22 29  1  6 15 66 98  4 88 25 60 64 23]
cars: {1, 3, 4, 6, 7, 9, 10, 12, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 33, 59, 60, 64, 66, 88, 98}
#cars= 33
Indy500: carno=33, lapnum=201
crash?:carno= 3
crash?:carno= 24
crash?:carno= 33
count of completed cars: 9
completed cars: [ 9 22 27  5 28 15 30 18 10]
cars: {5, 9, 10, 15, 18, 22, 27, 28, 30}
#cars= 9
count of completed cars: 9
completed cars: [ 9 22 27  5 28 15 30 18 10]
cars: {1, 3, 4, 5, 6, 7, 9, 10, 12, 14, 15, 18, 19, 20, 21, 22, 23, 25, 26, 27, 28, 30, 47, 55, 57, 59, 60, 68, 73, 83, 88, 98}
#cars= 32
Texas: carno=32, lapnum=249
crash?:carno= 12
crash?:carno= 19
count of completed cars: 5
complet

In [11]:
stagedf0, carindex = extract_features(stagedata)

/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/numpy/core/_methods.py:140: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/numpy/core/_methods.py:110: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


eventid=0, records=114
eventid=1, records=225
eventid=2, records=127
eventid=3, records=109
eventid=4, records=126
eventid=5, records=104


In [12]:
%debug

ERROR:root:No traceback has been produced, nothing to debug.


In [13]:
stagedf0.head(10)

,target,start_lap,eventid,car_number,stageid,firststage,pit_in_caution,start_position,start_rank,start_rank_ratio,...,laptime_std_all,laps_prev,laps_after_last_pitstop,pittime_prev,prev_nb0_change_in_rank,prev_nb1_change_in_rank,prev_nb2_change_in_rank,follow_nb0_change_in_rank,follow_nb1_change_in_rank,follow_nb2_change_in_rank
0,0,0,0,1,0,0,0,7,7,0.304348,...,0.000000,0,0,0.00000,0,0,0,0,0,0
1,-2,43,0,1,1,1,1,7,7,0.304348,...,6.431794,43,43,11.54325,0,0,0,0,0,0
2,6,119,0,1,2,1,0,7,5,0.217391,...,7.771622,76,76,59.63585,0,0,0,0,0,0
3,-7,176,0,1,3,1,0,7,11,0.478261,...,6.740054,57,57,40.43850,0,0,0,0,0,0
4,-3,232,0,1,4,1,1,7,4,0.173913,...,6.458577,56,56,39.51240,0,0,0,0,0,0
5,-4,0,0,4,0,0,0,16,16,0.695652,...,0.000000,0,0,0.00000,0,0,0,0,0,0
6,8,43,0,4,1,1,1,16,12,0.521739,...,6.433862,43,43,12.03580,0,0,0,0,0,0
7,-13,47,0,4,2,1,1,16,20,0.869565,...,11.236668,4,4,63.71120,0,0,0,0,0,0
8,15,118,0,4,3,1,0,16,7,0.304348,...,8.071575,71,71,48.16750,0,0,0,0,0,0
9,-2,123,0,4,4,1,0,16,22,0.956522,...,19.289386,5,5,126.40480,0,0,0,0,0,0


In [14]:
#stagedf0.reset_index(inplace=True)
stagedf = update_neighbor(stagedf0, carindex) 

In [15]:
%debug

ERROR:root:No traceback has been produced, nothing to debug.


In [16]:
stagedf = stagedf.drop(columns=['start_lap'])

In [27]:
suffix='withneighbor-newfeatures-timediff'
stagedf.to_csv('stage-%s-%s.csv'%(year,suffix))

In [18]:
stagedf

,target,eventid,car_number,stageid,firststage,pit_in_caution,start_position,start_rank,start_rank_ratio,top_pack,...,laptime_std_all,laps_prev,laps_after_last_pitstop,pittime_prev,prev_nb0_change_in_rank,prev_nb1_change_in_rank,prev_nb2_change_in_rank,follow_nb0_change_in_rank,follow_nb1_change_in_rank,follow_nb2_change_in_rank
0,0,0,1,0,0,0,7,7,0.304348,0,...,0.000000,0,0,0.00000,0,0,0,0,0,0
1,-2,0,1,1,1,1,7,7,0.304348,0,...,6.431794,43,43,11.54325,-2,-1,1,3,0,-1
2,6,0,1,2,1,0,7,5,0.217391,1,...,7.771622,76,76,59.63585,0,-2,7,-3,18,2
3,-7,0,1,3,1,0,7,11,0.478261,0,...,6.740054,57,57,40.43850,-12,-9,-18,-6,-7,4
4,-3,0,1,4,1,1,7,4,0.173913,1,...,6.458577,56,56,39.51240,-8,-2,-2,-4,2,-3
5,-4,0,4,0,0,0,16,16,0.695652,0,...,0.000000,0,0,0.00000,0,0,0,0,0,0
6,8,0,4,1,1,1,16,12,0.521739,0,...,6.433862,43,43,12.03580,-2,-1,0,-5,-3,-5
7,-13,0,4,2,1,1,16,20,0.869565,0,...,11.236668,4,4,63.71120,-1,3,-3,0,0,0
8,15,0,4,3,1,0,16,7,0.304348,0,...,8.071575,71,71,48.16750,0,-2,7,18,2,4
9,-2,0,4,4,1,0,16,22,0.956522,0,...,19.289386,5,5,126.40480,0,2,-12,0,0,0


In [19]:
            newfeatures=['laptime_green_mean_prev','laptime_green_std_prev','laptime_green_mean_all','laptime_green_std_all', 
             'laptime_mean_prev','laptime_std_prev','laptime_mean_all','laptime_std_all', 
             'laps_prev','laps_after_last_pitstop','pittime_prev'] 
            
stagedf[(stagedf['eventid']==1)][newfeatures].head(10)

,laptime_green_mean_prev,laptime_green_std_prev,laptime_green_mean_all,laptime_green_std_all,laptime_mean_prev,laptime_std_prev,laptime_mean_all,laptime_std_all,laps_prev,laps_after_last_pitstop,pittime_prev
114,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0.00000
115,40.213703,7.251773,40.213703,7.251773,40.213703,7.251773,40.213703,7.251773,32,32,21.18395
116,43.648931,6.920709,41.358779,7.324385,47.662917,14.816113,42.895420,11.201238,18,18,61.12735
117,49.960450,14.699757,42.587589,9.268853,77.241570,30.393279,52.708606,24.378173,20,20,119.20330
118,43.496911,4.780590,42.890696,8.066981,50.165945,20.982544,51.928185,23.417827,31,31,117.22760
119,42.725109,4.550299,42.842985,7.231663,42.725109,4.550299,49.610373,20.771467,34,34,59.83245
120,50.600871,13.147432,43.665791,8.413658,71.761266,23.992821,54.170851,23.267211,35,35,59.70915
121,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0.00000
122,40.214771,7.350604,40.214771,7.350604,40.214771,7.350604,40.214771,7.350604,31,31,21.64750
123,43.502394,6.716324,41.379138,7.303672,47.325416,14.155879,42.916816,11.025401,19,19,61.07185


In [20]:
np.max(stagedf)

target                        21.000000
eventid                        5.000000
car_number                    98.000000
stageid                       10.000000
firststage                     1.000000
pit_in_caution                 1.000000
start_position                33.000000
start_rank                    33.000000
start_rank_ratio               1.000000
top_pack                       1.000000
bottom_pack                    1.000000
average_rank                  33.000000
average_rank_all              33.000000
change_in_rank                21.000000
change_in_rank_all            15.000000
rate_of_change                28.000000
rate_of_change_all            10.500000
laptime_green_mean_prev      162.841500
laptime_green_std_prev       126.047271
laptime_green_mean_all        90.001400
laptime_green_std_all         60.384914
laptime_mean_prev            202.631767
laptime_std_prev             280.970336
laptime_mean_all             126.579100
laptime_std_all              280.970336


In [21]:
stagedf[stagedf['pittime_prev']>=3644][newfeatures]

,laptime_green_mean_prev,laptime_green_std_prev,laptime_green_mean_all,laptime_green_std_all,laptime_mean_prev,laptime_std_prev,laptime_mean_all,laptime_std_all,laps_prev,laps_after_last_pitstop,pittime_prev


In [22]:
chkdata = stagedata['Pocono'][1]

In [23]:
chkdata[(chkdata['car_number']==27) &(chkdata['lap_status']=='P')]

,rank,car_number,unique_id,completed_laps,elapsed_time,last_laptime,lap_status,best_laptime,best_lap,time_behind_leader,...,time_behind_prec,laps_behind_prec,overall_rank,overall_best_laptime,current_status,track_status,pit_stop_count,last_pitted_lap,start_position,laps_led
MyIdx,,,,,,,,,,,,,,,,,,,,,
506,1,27,2,8,777.5886,162.3380,P,45.6228,7,0.0000,...,0.0000,0,1,45.6228,Active,Y,1,8,3,2
7242,2,27,2,38,2370.4040,49.0508,P,41.9607,10,0.1960,...,0.1960,0,2,41.9607,Active,G,2,26,3,1F
9015,1,27,2,71,3801.7671,48.8333,P,41.7702,29,0.0000,...,0.0000,0,1,41.7702,Active,G,3,47,3,3C
10756,1,27,2,104,5240.7910,49.3872,P,41.7702,29,0.0000,...,0.0000,0,1,41.7702,Active,G,4,68,3,5C
12446,2,27,2,136,6634.1885,47.9170,P,41.6957,86,4.0873,...,4.0873,0,2,41.6957,Active,G,5,88,3,7B
14143,2,27,2,169,8078.3297,48.5346,P,41.6957,86,3.0713,...,3.0713,0,2,41.6957,Active,G,6,A9,3,97


In [24]:
chkdata[(chkdata['car_number']==27) &(chkdata['completed_laps']<15)]

,rank,car_number,unique_id,completed_laps,elapsed_time,last_laptime,lap_status,best_laptime,best_lap,time_behind_leader,...,time_behind_prec,laps_behind_prec,overall_rank,overall_best_laptime,current_status,track_status,pit_stop_count,last_pitted_lap,start_position,laps_led
MyIdx,,,,,,,,,,,,,,,,,,,,,
2,3,27,2,0,0.1156,99.7775,T,0.0000,0,0.1156,...,0.0000,0,3,0.0000,Active,G,0,0,3,0
45,2,27,2,1,78.3457,78.2301,T,78.2301,1,0.7180,...,0.7180,0,2,78.2301,Active,Y,0,0,3,0
112,2,27,2,2,182.7721,104.4264,T,78.2301,1,0.6347,...,0.6347,0,2,78.2301,Active,Y,0,0,3,0
168,2,27,2,3,287.8186,105.0465,T,78.2301,1,0.6086,...,0.6086,0,2,78.2301,Active,Y,0,0,3,0
225,2,27,2,4,387.3480,99.5294,T,78.2301,1,1.1202,...,1.1202,0,2,78.2301,Active,Y,0,0,3,0
288,2,27,2,5,478.0215,90.6735,T,78.2301,1,0.5450,...,0.5450,0,2,78.2301,Active,Y,0,0,3,0
326,2,27,2,6,569.6278,91.6063,T,78.2301,1,0.0639,...,0.0639,0,2,78.2301,Active,G,0,0,3,0
366,1,27,2,7,615.2506,45.6228,T,45.6228,7,0.0000,...,0.0000,0,1,45.6228,Active,Y,0,0,3,1
506,1,27,2,8,777.5886,162.3380,P,45.6228,7,0.0000,...,0.0000,0,1,45.6228,Active,Y,1,8,3,2


In [25]:
#check the integrity of the data(laptime)
#for event in stagedata.keys():
#    alldata, rankdata, acldata, flagdata, stage, data = stagedata[event]
    
chkdata = stagedata['Pocono'][2]
chkdata[chkdata['car_number']==27].head(10)

,car_number,completed_laps,rank,elapsed_time,rank_diff,time_diff
MyIdx,,,,,,
2,27,0,3,0.1156,0.0,0.0000
45,27,1,2,78.3457,-1.0,78.2301
112,27,2,2,182.7721,0.0,104.4264
168,27,3,2,287.8186,0.0,105.0465
225,27,4,2,387.3480,0.0,99.5294
288,27,5,2,478.0215,0.0,90.6735
326,27,6,2,569.6278,0.0,91.6063
366,27,7,1,615.2506,-1.0,45.6228
506,27,8,1,777.5886,0.0,162.3380
